In [1]:
#Importing required libraries for data cleanup on CSV files, and to create DataFrames

import pandas as pd
from pathlib import Path
import numpy as np
import csv
from datetime import datetime

In [2]:
#Creating Dataframe from "inlaw_portfolio" CSV file

inlaw_df = pd.read_csv(Path("Resources/inlaw_portfolio_032021_to_050821.csv"))

In [3]:
print(inlaw_df.info())
display(inlaw_df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Date               398 non-null    object 
 1   Type               398 non-null    object 
 2   Action             386 non-null    object 
 3   Symbol             386 non-null    object 
 4   Instrument Type    386 non-null    object 
 5   Description        398 non-null    object 
 6   Value              398 non-null    object 
 7   Quantity           398 non-null    int64  
 8   Average Price      386 non-null    object 
 9   Commissions        398 non-null    object 
 10  Fees               398 non-null    float64
 11  Multiplier         350 non-null    float64
 12  Underlying Symbol  350 non-null    object 
 13  Expiration Date    350 non-null    object 
 14  Strike Price       350 non-null    float64
 15  Call or Put        350 non-null    object 
 16  Order #            383 non

,Date,Type,Action,Symbol,Instrument Type,Description,Value,Quantity,Average Price,Commissions,Fees,Multiplier,Underlying Symbol,Expiration Date,Strike Price,Call or Put,Order #
0,2021-05-08T06:30:42-0700,Money Movement,NaN,NaN,NaN,Regulatory fee adjustment,-0.01,0,NaN,--,0.000,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-07T12:13:35-0700,Trade,SELL_TO_OPEN,IQ 210521P00013000,Equity Option,Sold 1 IQ 05/21/21 Put 13.00 @ 0.49,49.00,1,49.00,-1.00,-0.142,100.0,IQ,5/21/21,13.0,PUT,148294915.0
2,2021-05-07T09:52:54-0700,Trade,SELL_TO_OPEN,./ESM1 ESM1 210618C4420,Future Option,Sold 1 /ESM1 ESM1 06/18/21 Call 4420.00 @ 10.25,512.50,1,512.50,-2.50,-0.870,1.0,/ESM1,6/18/21,4420.0,CALL,148228548.0
3,2021-05-07T09:52:41-0700,Trade,BUY_TO_CLOSE,./ESM1 EW3K1 210521C4300,Future Option,Bought 1 /ESM1 EW3K1 05/21/21 Call 4300.00 @ 9.0,-450.00,1,-450.00,0.00,-0.870,1.0,/ESM1,5/21/21,4300.0,CALL,148227829.0
4,2021-05-07T09:44:31-0700,Trade,BUY_TO_CLOSE,SPY 210521C00421000,Equity Option,Bought 2 SPY 05/21/21 Call 421.00 @ 4.56,-912.00,2,-456.00,0.00,-0.270,100.0,SPY,5/21/21,421.0,CALL,148224512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,2021-03-12T10:07:46-0800,Trade,SELL_TO_OPEN,TLRY 210416P00020000,Equity Option,Sold 1 TLRY 04/16/21 Put 20.00 @ 1.98,198.00,1,198.00,-1.00,-0.152,100.0,TLRY,4/16/21,20.0,PUT,137206787.0
394,2021-03-12T10:07:46-0800,Trade,SELL_TO_OPEN,TLRY 210416C00070000,Equity Option,Sold 1 TLRY 04/16/21 Call 70.00 @ 1.92,192.00,1,192.00,-1.00,-0.152,100.0,TLRY,4/16/21,70.0,CALL,137206787.0
395,2021-03-12T08:20:26-0800,Trade,SELL_TO_OPEN,./ESM1 EWJ1 210430P3170,Future Option,Sold 1 /ESM1 EWJ1 04/30/21 Put 3170.00 @ 11.5,575.00,1,575.00,-2.50,-0.870,1.0,/ESM1,4/30/21,3170.0,PUT,137149650.0
396,2021-03-12T02:30:56-0800,Trade,SELL_TO_OPEN,./ESM1 EWJ1 210430C4150,Future Option,Sold 1 /ESM1 EWJ1 04/30/21 Call 4150.00 @ 17.5,875.00,1,875.00,-2.50,-0.870,1.0,/ESM1,4/30/21,4150.0,CALL,137025523.0


In [4]:
#Replace the "T" with a space to better format for datetime

inlaw_df['Date'] = inlaw_df['Date'].str.replace('T','  ')
display(inlaw_df['Date'])

0      2021-05-08  06:30:42-0700
1      2021-05-07  12:13:35-0700
2      2021-05-07  09:52:54-0700
3      2021-05-07  09:52:41-0700
4      2021-05-07  09:44:31-0700
                 ...            
393    2021-03-12  10:07:46-0800
394    2021-03-12  10:07:46-0800
395    2021-03-12  08:20:26-0800
396    2021-03-12  02:30:56-0800
397    2021-03-11  14:00:00-0800
Name: Date, Length: 398, dtype: object

In [5]:
#Convert the Date column into datetime datatype

inlaw_df['Date'] = pd.to_datetime(inlaw_df['Date'], utc = True)
display(inlaw_df['Date'])

0     2021-05-08 13:30:42+00:00
1     2021-05-07 19:13:35+00:00
2     2021-05-07 16:52:54+00:00
3     2021-05-07 16:52:41+00:00
4     2021-05-07 16:44:31+00:00
                 ...           
393   2021-03-12 18:07:46+00:00
394   2021-03-12 18:07:46+00:00
395   2021-03-12 16:20:26+00:00
396   2021-03-12 10:30:56+00:00
397   2021-03-11 22:00:00+00:00
Name: Date, Length: 398, dtype: datetime64[ns, UTC]

In [6]:
#Drop the time from the Date column

inlaw_df['Date'] = pd.to_datetime(inlaw_df['Date']).dt.date
display(inlaw_df['Date'])

0      2021-05-08
1      2021-05-07
2      2021-05-07
3      2021-05-07
4      2021-05-07
          ...    
393    2021-03-12
394    2021-03-12
395    2021-03-12
396    2021-03-12
397    2021-03-11
Name: Date, Length: 398, dtype: object

In [7]:
#Set the index of the Dateframe to the Date column

inlaw_df = inlaw_df.set_index('Date')
display(inlaw_df)

,Type,Action,Symbol,Instrument Type,Description,Value,Quantity,Average Price,Commissions,Fees,Multiplier,Underlying Symbol,Expiration Date,Strike Price,Call or Put,Order #
Date,,,,,,,,,,,,,,,,
2021-05-08,Money Movement,NaN,NaN,NaN,Regulatory fee adjustment,-0.01,0,NaN,--,0.000,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-07,Trade,SELL_TO_OPEN,IQ 210521P00013000,Equity Option,Sold 1 IQ 05/21/21 Put 13.00 @ 0.49,49.00,1,49.00,-1.00,-0.142,100.0,IQ,5/21/21,13.0,PUT,148294915.0
2021-05-07,Trade,SELL_TO_OPEN,./ESM1 ESM1 210618C4420,Future Option,Sold 1 /ESM1 ESM1 06/18/21 Call 4420.00 @ 10.25,512.50,1,512.50,-2.50,-0.870,1.0,/ESM1,6/18/21,4420.0,CALL,148228548.0
2021-05-07,Trade,BUY_TO_CLOSE,./ESM1 EW3K1 210521C4300,Future Option,Bought 1 /ESM1 EW3K1 05/21/21 Call 4300.00 @ 9.0,-450.00,1,-450.00,0.00,-0.870,1.0,/ESM1,5/21/21,4300.0,CALL,148227829.0
2021-05-07,Trade,BUY_TO_CLOSE,SPY 210521C00421000,Equity Option,Bought 2 SPY 05/21/21 Call 421.00 @ 4.56,-912.00,2,-456.00,0.00,-0.270,100.0,SPY,5/21/21,421.0,CALL,148224512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-12,Trade,SELL_TO_OPEN,TLRY 210416P00020000,Equity Option,Sold 1 TLRY 04/16/21 Put 20.00 @ 1.98,198.00,1,198.00,-1.00,-0.152,100.0,TLRY,4/16/21,20.0,PUT,137206787.0
2021-03-12,Trade,SELL_TO_OPEN,TLRY 210416C00070000,Equity Option,Sold 1 TLRY 04/16/21 Call 70.00 @ 1.92,192.00,1,192.00,-1.00,-0.152,100.0,TLRY,4/16/21,70.0,CALL,137206787.0
2021-03-12,Trade,SELL_TO_OPEN,./ESM1 EWJ1 210430P3170,Future Option,Sold 1 /ESM1 EWJ1 04/30/21 Put 3170.00 @ 11.5,575.00,1,575.00,-2.50,-0.870,1.0,/ESM1,4/30/21,3170.0,PUT,137149650.0


In [8]:
#Creating new Dataframe to hold Money Movement types from inlaw_df for additional metrics later

inlaw_money_movement_df = pd.DataFrame()
display(inlaw_money_movement_df)

""


In [9]:
#Filling new inlaw_money_movement_df with Money Movement types from inlaw_df

inlaw_money_movement_df = inlaw_df[(inlaw_df['Type'] == 'Money Movement')]

In [10]:
display(inlaw_money_movement_df)

,Type,Action,Symbol,Instrument Type,Description,Value,Quantity,Average Price,Commissions,Fees,Multiplier,Underlying Symbol,Expiration Date,Strike Price,Call or Put,Order #
Date,,,,,,,,,,,,,,,,
2021-05-08,Money Movement,NaN,NaN,NaN,Regulatory fee adjustment,-0.01,0,NaN,--,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-05-01,Money Movement,NaN,NaN,NaN,Regulatory fee adjustment,-0.10,0,NaN,--,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-24,Money Movement,NaN,NaN,NaN,Regulatory fee adjustment,-0.07,0,NaN,--,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-17,Money Movement,NaN,NaN,NaN,Regulatory fee adjustment,-0.09,0,NaN,--,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-16,Money Movement,NaN,NaN,NaN,INTEREST ON CREDIT BALANCE,2.42,0,NaN,--,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-10,Money Movement,NaN,NaN,NaN,Regulatory fee adjustment,-0.20,0,NaN,--,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-04-08,Money Movement,NaN,NaN,NaN,ACH DISBURSEMENT,"-2,500.00",0,NaN,--,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-27,Money Movement,NaN,NaN,NaN,Regulatory fee adjustment,-0.11,0,NaN,--,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2021-03-20,Money Movement,NaN,NaN,NaN,Regulatory fee adjustment,-0.17,0,NaN,--,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#Drop null values from the Dataframe

inlaw_df = inlaw_df.dropna()
display(inlaw_df)

,Type,Action,Symbol,Instrument Type,Description,Value,Quantity,Average Price,Commissions,Fees,Multiplier,Underlying Symbol,Expiration Date,Strike Price,Call or Put,Order #
Date,,,,,,,,,,,,,,,,
2021-05-07,Trade,SELL_TO_OPEN,IQ 210521P00013000,Equity Option,Sold 1 IQ 05/21/21 Put 13.00 @ 0.49,49.00,1,49.00,-1.00,-0.142,100.0,IQ,5/21/21,13.0,PUT,148294915.0
2021-05-07,Trade,SELL_TO_OPEN,./ESM1 ESM1 210618C4420,Future Option,Sold 1 /ESM1 ESM1 06/18/21 Call 4420.00 @ 10.25,512.50,1,512.50,-2.50,-0.870,1.0,/ESM1,6/18/21,4420.0,CALL,148228548.0
2021-05-07,Trade,BUY_TO_CLOSE,./ESM1 EW3K1 210521C4300,Future Option,Bought 1 /ESM1 EW3K1 05/21/21 Call 4300.00 @ 9.0,-450.00,1,-450.00,0.00,-0.870,1.0,/ESM1,5/21/21,4300.0,CALL,148227829.0
2021-05-07,Trade,BUY_TO_CLOSE,SPY 210521C00421000,Equity Option,Bought 2 SPY 05/21/21 Call 421.00 @ 4.56,-912.00,2,-456.00,0.00,-0.270,100.0,SPY,5/21/21,421.0,CALL,148224512.0
2021-05-07,Trade,SELL_TO_OPEN,SPY 210618C00428000,Equity Option,Sold 2 SPY 06/18/21 Call 428.00 @ 4.61,922.00,2,461.00,-2.00,-0.284,100.0,SPY,6/18/21,428.0,CALL,148224512.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-12,Trade,SELL_TO_OPEN,SPCE 210416P00025000,Equity Option,Sold 1 SPCE 04/16/21 Put 25.00 @ 1.12,112.00,1,112.00,-1.00,-0.152,100.0,SPCE,4/16/21,25.0,PUT,137210922.0
2021-03-12,Trade,SELL_TO_OPEN,TLRY 210416P00020000,Equity Option,Sold 1 TLRY 04/16/21 Put 20.00 @ 1.98,198.00,1,198.00,-1.00,-0.152,100.0,TLRY,4/16/21,20.0,PUT,137206787.0
2021-03-12,Trade,SELL_TO_OPEN,TLRY 210416C00070000,Equity Option,Sold 1 TLRY 04/16/21 Call 70.00 @ 1.92,192.00,1,192.00,-1.00,-0.152,100.0,TLRY,4/16/21,70.0,CALL,137206787.0
